In [55]:
# Run this cell to import the neccessary imports.
# Make sure you have the following files in your working directory
#        "inputs.txt"
#        "relu_bias.txt"
#        "relu_weights.txt"
#        "sigmoid_bias.txt"
#        "sigmoid_weights.txt"

import numpy as np
import os

# input binaries
in_bin_folder = './input_binaries'
if not os.path.exists(in_bin_folder):
    os.makedirs(in_bin_folder)

# relu package sv file folder
relu_pkg_folder = './relu_weights_biases_pkgs'
if not os.path.exists(relu_pkg_folder):
    os.makedirs(relu_pkg_folder)

# sigmoid package sv file folder
sig_pkg_folder = './sigmoid_weights_biases_pkgs'
if not os.path.exists(sig_pkg_folder):
    os.makedirs(sig_pkg_folder)

# binary that goes to sigmoid bram/look up table
sbram_bin_folder = './sigmoid_BRAM_binaries'
if not os.path.exists(sbram_bin_folder):
    os.makedirs(sbram_bin_folder)



def flip_bits(val, bits):
    return ((val ^ (2**bits - 1)) + 1)

def sigmoid(x):
    return 1/(1+(np.e**(-x)))

# really innefficient but calling "sed" subprocess didn't work
# Removes negative sign "-" from file
def remove_negs(tmp_file, dest_file_name):
    output_file = open(tmp_file, 'r')
    dest_file = open(dest_file_name, 'w')
    file = output_file.read()
    file = file.replace('b-', 'b')
    dest_file.write(file)
    # close files
    dest_file.close()
    output_file.close()
    
# =========================================================================
# pkg functions (for sub-processes)
# =========================================================================
def get_pkg_name(name, width, nfrac):
    return str(name) + str(width) + '_' + str(nfrac)

def conv_to_str(num, width, nfrac):
    div = (2**nfrac)*1.0
    x = round(float(num * div // 1))
    if (x < 0):
        return format(flip_bits(x, width), '0'+ str(width) + 'b')
    return format(x, '0'+ str(width) + 'b')


# =========================================================================


# ALter this to reflect state of files
NUM_INPUTS = 10
NUM_BIAS_RELU = 32
NUM_WEIGHTS_RELU = 320
NUM_BIAS_SIGMOID = 1
NUM_WEIGHTS_SIGMOID = 32

In [56]:
# =======================================================
# generate input vals
# =======================================================

def generate_input_bin_file(width, nfrac):
    div = (2**nfrac)*1.0

    src_file = open("inputs.txt", 'r')
    
    file_name = in_bin_folder + "/bw" + str(width) + "_nfrac" + str(nfrac) + "_inputs.mem"
    tmp_file = in_bin_folder + "/tmp"
    dest_file = open(tmp_file, 'w')


    for i in range(NUM_INPUTS):
        x = float(src_file.readline())
        dest_file.write(conv_to_str(x, width, nfrac))
        dest_file.write("\n")
    
    #subprocess.call(["sed", 's/-//g', file_name]) # doesn't work
    dest_file.close()
    src_file.close()
    
    # replace negative '-' character
    remove_negs(tmp_file, file_name)

# =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# Set width and fractional bits you want to generate
width = 10
nfrac = 5

# run function
# generate_input_bin_file(width, nfrac)


In [57]:
# =======================================================
# generate relu wights and bias package
# =======================================================


def generate_relu_pkg_file(width, nfrac):
    # open source files
    src_weights_file = open("relu_weights.txt", 'r')
    src_biases_file = open("relu_bias.txt", 'r')
    
    # make the final output files
    file_name = relu_pkg_folder + "/" + get_pkg_name("relu", width, nfrac) + ".sv"
    tmp_file = relu_pkg_folder + "/tmp"
    dest_file = open(tmp_file, 'w') # write to tmp file, will write to final when cleaning up '-' signs

    # first decalaration lines
    dest_file.write("// Width: " + str(width) + "\n// NFRAC: " + str(nfrac) + "\n")
    dest_file.write("package " + str(get_pkg_name("relu", width, nfrac)) + ';\n\n')
    dest_file.write("localparam logic signed ["+ str(width-1) + ":0] weights [" + str(NUM_WEIGHTS_RELU) + "] = \'{\n")

    # convert weights (from floats) into binary
    for i in range(NUM_WEIGHTS_RELU):
        flt_num = float(src_weights_file.readline())
        
        # e.g. write line "17'b01011011011111001,"
        dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
        if (i != (NUM_WEIGHTS_RELU-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(",  // " + str(flt_num) + "\n")
        else:
            dest_file.write("   // " + str(flt_num) + "\n")
    
    # switch to declaring biases
    dest_file.write("};\n\n")
    dest_file.write("localparam logic signed ["+ str(width-1) + ":0] bias [" + str(NUM_BIAS_RELU) + "] = \'{\n")
    
    # convert bias (from floats) into binary
    for i in range(NUM_BIAS_RELU):
        flt_num = float(src_biases_file.readline())
        
        # e.g. write line "17'b01011011011111001,"
        dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
        if (i != (NUM_BIAS_RELU-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(",  // " + str(flt_num) + "\n")
        else:
            dest_file.write("   // " + str(flt_num) + "\n")
        
        
    # finish off declarations
    dest_file.write("};\nendpackage")
    
    
    
    #subprocess.call(["sed", 's/-//g', file_name]) # doesn't work
    dest_file.close()
    src_biases_file.close()
    src_weights_file.close()
    
    # replace negative '-' character
    remove_negs(tmp_file, file_name)


# =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# Set width and fractional bits you want to generate
width = 10
nfrac = 5

# run function
# generate_relu_pkg_file(width, nfrac)

In [58]:
# =======================================================
# generate sigmoid wights and bias package
# =======================================================


def generate_sigmoid_pkg_file(width, nfrac):
    # open source files
    src_weights_file = open("sigmoid_weights.txt", 'r')
    src_biases_file = open("sigmoid_bias.txt", 'r')
    
    # make the final output files
    file_name = sig_pkg_folder + "/" + get_pkg_name("sigmoid", width, nfrac) + ".sv"
    tmp_file = sig_pkg_folder + "/tmp"
    dest_file = open(tmp_file, 'w') # write to tmp file, will write to final when cleaning up '-' signs

    # first decalaration lines
    dest_file.write("// Width: " + str(width) + "\n// NFRAC: " + str(nfrac) + "\n")
    dest_file.write("package " + str(get_pkg_name("sigmoid", width, nfrac)) + ';\n\n')
    dest_file.write("localparam logic signed ["+ str(width-1) + ":0] weights [" + str(NUM_WEIGHTS_SIGMOID) + "] = \'{\n")

    # convert weights (from floats) into binary
    for i in range(NUM_WEIGHTS_SIGMOID):
        flt_num = float(src_weights_file.readline())
        
        # e.g. write line "17'b01011011011111001,"
        dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
        if (i != (NUM_WEIGHTS_SIGMOID-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(",  // " + str(flt_num) + "\n")
        else:
            dest_file.write("   // " + str(flt_num) + "\n")
    
    # switch to declaring biases
    dest_file.write("};\n\n")
    dest_file.write("localparam logic signed ["+ str(width-1) + ":0] bias [" + str(NUM_BIAS_SIGMOID) + "] = \'{\n")
    
    # convert bias (from floats) into binary
    for i in range(NUM_BIAS_SIGMOID):
        flt_num = float(src_biases_file.readline())
        
        # e.g. write line "17'b01011011011111001,"
        dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
        
        if (i != (NUM_BIAS_SIGMOID-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(",  // " + str(flt_num) + "\n")
        else:
            dest_file.write("   // " + str(flt_num) + "\n")
    
    # finish off declarations
    dest_file.write("};\nendpackage")
    
    
    
    #subprocess.call(["sed", 's/-//g', file_name]) # doesn't work
    dest_file.close()
    src_biases_file.close()
    src_weights_file.close()
    
    # replace negative '-' character
    remove_negs(tmp_file, file_name)


# =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# Set width and fractional bits you want to generate
width = 10
nfrac = 5

# run function
# generate_sigmoid_pkg_file(width, nfrac)

In [59]:
# =======================================================
# generate sigmoid bram
# =======================================================


def generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec):
    file = open("./sigmoid_BRAM_vals.txt", 'w')
    
    # number of table entries
    N_TABLE = 2**table_size_pow

    # initialize array with zeros
    index = [0] * N_TABLE

    for i in range(N_TABLE):
        index[i] = 16.0*(i-(float(N_TABLE)/2.0))/(float(N_TABLE))

    for n in index:
        file.write(str(sigmoid(n)))
        file.write("\n")

    file.close()

    # =================================================================
    # now convert into binary
    div = (2**prec)*1.0

    src_file = open("sigmoid_BRAM_vals.txt", 'r')
    
    file_name = sbram_bin_folder + "/memw" + str(prec) + "_tsize" + str(int(N_TABLE)) + "_sigmoidBRAM.mem"
    tmp_file = sbram_bin_folder + "/tmp.mem"
    dest_file = open(tmp_file, 'w')

    for i in range(int(N_TABLE)):
        x = round(float(src_file.readline())*div // 1)
    
        if (x < 0):
            dest_file.write(format(flip_bits(x, width), '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        else:
            dest_file.write(format(x, '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        
    dest_file.close()
    src_file.close()
    
    # replace negative '-' character
    remove_negs(tmp_file, file_name)


# =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# Set width and fractional bits you want to generate
width = 10
nfrac = 5

# run function
#generate_sigmoid_bram_bin_file(width, nfrac)



In [60]:
# ===============================================================================================
# ===============================================================================================
# ===============================================================================================
# ===============================================================================================

In [62]:

# generate specific set or write a simple for loop to generate multiple sets of files

width = 17              # width of numbers
nfrac = 10              # number of fractional bits
table_size_pow = 10     # BRAM size (2**table_size_pow)
prec = 10               # BRAM precision

# generate_input_bin_file(width, nfrac)
# generate_relu_pkg_file(width, nfrac)
# generate_sigmoid_pkg_file(width, nfrac)
# generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec)

# generate set of files for bit width 4-32
for i in range(4,33):
    width = i;
    nfrac = (i+1) // 2
    
    generate_input_bin_file(width, nfrac)
    generate_relu_pkg_file(width, nfrac)
    generate_sigmoid_pkg_file(width, nfrac)
    
generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec)

